In [ ]:
import pandas as pd
import seaborn as sb
import numpy as np
import os
import matplotlib.pyplot as mp
from scipy.stats import binom, nbinom, t, f , norm, poisson, chi2
from dash import Dash, html, dcc, callback, Input, Output
import plotly.express as px
import plotly.graph_objs as go

In [ ]:
# rvs = randon variate sample, vai gerar numeros aleatorios de acordo com uma distribuição
# fazer usando orientação a objetos, funcoes 'def', ou usando apenas 'IFs'?
# usar seaborn que é mais fácil ou matplotlib que é mais elaborado?
# usar 'Dash' para storytelling? 'pygame'? precisa-se de algo similar ao Rshiny
# fazer arquivo executável ou um link web?


In [ ]:
# construindo um dashboard online
# https://dash.plotly.com
# http://127.0.0.1:8050
app = Dash(__name__)
app.layout = html.Div(
    [
      
    html.H1 ('Distribuição estatística'),
    html.H2 ('Parâmetros'),
    
    'Tipo: ',
    dcc.Input(id = 'tipo', value = ' ', type = 'text'),
    html.Br(),
    
                            
    'Média: ',
    dcc.Input(id = 'media', value = ' ', type = 'number'),
    html.Br(),
    
    
    'Desvio-padrão: ',
    dcc.Input(id = 'desv_pad', value = ' ', type = 'number'),
    html.Br(),
    
    
    'Tamanho: ',
    dcc.Input(id = 'tamanho', value = ' ', type = 'number'),                    
    html.Br(),
    html.Br(),
    html.Br(),
    
    
    
    html.Button('Plote o gráfico', id='botao_grafico'),
    html.Br(),
    html.Br(),
    html.Br(),
    
    html.Div(id = 'my_output'),
    
    'Histograma',    
    dcc.Graph (id = 'grafico_distribuicao')
    
    ]
)
  

                       
 
@callback (
    Output(component_id='grafico_distribuicao', component_property='figure'),  # lembrar que sempre os Outputs vem primeiro e o callback sempre precisa estar acima da funcao que vai fazer o calculo , no caso é a 'gerar_output'
    Output(component_id='my_output', component_property='children'),
    Input(component_id='botao_grafico', component_property='n_clicks'),
    Input(component_id='tipo', component_property='value'),
    Input('media', 'value'),
    Input('desv_pad', 'value'),
    Input('tamanho', 'value'),
)

def botao_grafico ( tipo, media, desv_pad, tamanho):
    if tipo == 'normal' and media and desv_pad and tamanho:
        dist_nornm = norm(media, desv_pad)
        amostra = dist_nornm.rvs(size = tamanho)
        
        dados = go.Histogram(x = amostra, histnorm='probability')
        
        layout2 = go.Layout(
            title = 'Distribuição',
            xaxis = 'Valor',
            yaxis = 'Probabilidade'
            )
            
        
        
        grafico = go.Figure(data = dados, layout = layout2)
        
          
        return grafico  
             

if __name__ =='__main__':
    app.run_server(debug = True) # debug = True significa que o app ja atualiza automaticamente

In [ ]:
# exemplo bai chat
# import dash
# import dash_core_components as dcc
# import dash_html_components as html
# from dash.dependencies import Input, Output
# import plotly.graph_objs as go
# from scipy.stats import norm

# app = dash.Dash(__name__)

# app.layout = html.Div(
#     [
#         html.H1('Distribuição estatística'),
#         html.H2('Parâmetros'),
#         'Tipo: ',
#         dcc.Input(id='tipo', value='', type='text'),
#         html.Br(),
#         'Média: ',
#         dcc.Input(id='media', value='', type='number'),
#         html.Br(),
#         'Desvio-padrão: ',
#         dcc.Input(id='desv_pad', value='', type='number'),
#         html.Br(),
#         'Tamanho: ',
#         dcc.Input(id='tamanho', value='', type='number'),
#         html.Br(),
#         html.Button('Gerar gráfico', id='gerar_grafico'),
#         html.Div(id='my_output'),
#         dcc.Graph(id='grafico_distribuicao')
#     ]
# )

# @app.callback(
#     Output(component_id='grafico_distribuicao', component_property='figure'),
#     Output(component_id='my_output', component_property='children'),
#     Input('gerar_grafico', 'n_clicks'),
#     Input('tipo', 'value'),
#     Input('media', 'value'),
#     Input('desv_pad', 'value'),
#     Input('tamanho', 'value'),
# )
# def gerar_grafico(n_clicks, tipo, media, desv_pad, tamanho):
#     if n_clicks is None:
#         return go.Figure(), None
#     if tipo == 'normal' and media and desv_pad and tamanho:
#         dist_normal = norm(float(media), float(desv_pad))
#         amostra = dist_normal.rvs(size=int(tamanho))

#         histograma = go.Histogram(x=amostra, histnorm='probability')

#         layout = go.Layout(
#             title="Distribuição da amostra",
#             xaxis=dict(title='Valor'),
#             yaxis=dict(title='Probabilidade')
#         )

#         figura = go.Figure(data=[histograma], layout=layout)

#         return figura, None
#     else:
#         return go.Figure(), html.Div('Por favor, preencha todos os campos.')


# if __name__ == '__main__':
#     app.run_server(debug=True)

In [ ]:
# classe distribuição (faz-se a amostra com determinados parametros e plota grafico)
class Distribuicao:
    def __init__(self, tipo:str,
                 n_tentativas:int = None,
                 prob_sucesso:float = None,
                 tamanho:float = None,
                 n_sucessos:float = None,
                 media:float = None,
                 desv_pad:float = None,
                 graus_liberade:int = None,
                 graus_liberade_numerador:int = None,
                 graus_liberade_denominador:int = None):
        self.tipo = tipo
        self.n_tentativas = n_tentativas
        self.prob_sucesso = prob_sucesso
        self.tamanho = tamanho
        self.n_sucessos = n_sucessos
        self.media = media
        self.desv_pad = desv_pad
        self.graus_liberade = graus_liberade
        self.graus_liberade_numerador = graus_liberade_numerador
        self.graus_liberade_denominador = graus_liberade_denominador
        
    
    def gerar_amostra(self):
        
        # 3 prametros: numero de tentativas, prob de sucesso, tamanho da amostra
        if self.tipo == 'binomial':
            dist_binomial = binom(self.n_tentativas, self.prob_sucesso)
            amostra = dist_binomial.rvs(size = self.tamanho)
            print(amostra)
            grafico = sb.displot(amostra)
            # mp.hist(amostra, bins=10)
            # mp.xlabel('valores')
            # mp.ylabel('frequencia')
            
            print(f'essa distribuicao é {self.tipo}')
        

        # 3 parametros: numero de sucessos, prob de sucesso, tamanho da amostra
        elif self.tipo == 'binomial_negativa':            
            dist_nbinomial = nbinom(self.n_sucessos, self.prob_sucesso)
            amostra = dist_nbinomial.rvs(size = self.tamanho)
            print(amostra)
            grafico = sb.displot(amostra)
            

            print(f'essa distribuicao é {self.tipo}')

        
        # 2 parametros: media e tamanho da amostra        
        elif self.tipo == 'poisson':            
            dist_poisson = poisson(self.media)
            amostra = dist_poisson.rvs(size = self.tamanho)
            print(amostra)
            grafico = sb.displot(amostra)

            print(f'essa distribuicao é {self.tipo}')
        
        
        # 3 parametros: media, desvio-padrao, tamanho da amostra
        elif self.tipo == 'normal':            
            dist_normal = norm(loc = self.media, scale = self.desv_pad )
            amostra = dist_normal.rvs(size = self.tamanho)
            print(amostra)
            grafico = sb.displot(amostra)

            print(f'essa distribuicao é {self.tipo}')
            
        
        
        # 2 parametros: graus de liberdade e tamanho da amostra
        elif self.tipo == 'qui2':            
            dist_qui2 = chi2(self.graus_liberade)
            amostra = dist_qui2.rvs(size = self.tamanho)
            print(amostra)
            grafico = sb.displot(amostra)

            print(f'essa distribuicao é {self.tipo}')
            
            
            
        # 2 parametros: graus de liberdade e tamanho da amostra
        elif self.tipo == 't':            
            dist_t = t(self.graus_liberade)
            amostra = dist_t.rvs(size = self.tamanho)
            print(amostra)
            grafico = sb.displot(amostra)

            print(f'essa distribuicao é {self.tipo}')
            
            
            
        # 3 parametros: graus de liberdade numerador e denominador e tamanho da amostra
        elif self.tipo == 'f':            
            dist_f = f(self.graus_liberade_numerador, self.graus_liberade_denominador)
            amostra = dist_f.rvs(size = self.tamanho)
            print(amostra)
            grafico = sb.displot(amostra)

            print(f'essa distribuicao é {self.tipo}')           
            
        
           

In [ ]:
distribuicao = Distribuicao(tipo = 'normal', media = 5,desv_pad=1, tamanho = 1000)
distribuicao.gerar_amostra()


app.layout = html.Div( children = 
                      [html.H1 (children = 'Distribuição estatística selecionada'),
                       html.H2 (children = 'teste'),
                       dcc.Graph(id='teste', figure = distribuicao.gerar_amostra())])
                       
 

In [ ]:
# CODIGO BONITAO
# class Distribuicao:
#     def __init__(self, tipo: str, **kwargs):
#         self.tipo = tipo
#         self.kwargs = kwargs
    
#     def gerar_amostra(self):
#         distribuicoes = {
#             'binomial': (binom, ['n', 'p']),
#             'binomial_negativa': (nbinom, ['n', 'p']),
#             'poisson': (poisson, ['mu']),
#             'normal': (norm, ['loc', 'scale']),
#             'qui2': (chi2, ['df']),
#             't': (t, ['df']),
#             'f': (f, ['dfn', 'dfd'])
#         }
        
#         dist_func, parametros = distribuicoes[self.tipo]
#         dist_args = {parametro: self.kwargs[parametro] for parametro in parametros}
        
#         dist = dist_func(**dist_args)
#         amostra = dist.rvs(size=self.kwargs.get('tamanho', 1))
        
#         print(amostra)
#         sb.displot(amostra)

#         print(f"Essa distribuição é {self.tipo}")